In [1]:
#Author: ML Tlachac, WPI
#For StudentSADD, 2021

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from scipy import stats
import collections
import operator
import argparse
import random
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn import metrics
from statistics import mean 
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import resample
from sklearn import preprocessing
from sklearn import utils
from sklearn.datasets import load_digits
from sklearn import svm
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.decomposition import PCA, KernelPCA, NMF

C:\Users\mltla\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\mltla\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\mltla\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
#data = pd.read_csv("scriptedML.csv")
#print(data.shape)
#data.head()

In [13]:
data = pd.read_csv("tPromptTextFeatures.csv")
#unscriptedML.csv
#aPromptTextFeatures.csv
#tPromptTextFeatures.csv
print(data.shape)

#binary labels
d10 = []
s1 = []
for i in range(0, data.shape[0]):
    if int(data.phq9[i]) >= 10:
        d10.append(1)
    else:
        d10.append(0)
    if int(data.q9[i]) >= 1:
        s1.append(1)
    else:
        s1.append(0)
data["phq9"] = d10
data["q9"] = s1
data.head()

data.head()

(341, 208)


,id,phq9,q9,Characters,Words,help,office,dance,money,wedding,...,musical,colors,text_abbreviations,NN,PRP,CD,DT,SYM,NNS,NNP
0,1607087749,1,0,1950,355,0.000000,0.005634,0.0,0.000000,0.002817,...,0.0,0.002817,0.0,0.890089,0.003812,0.00000,0.087039,0.0,0.019060,0.0
1,6831,0,0,1483,295,0.003401,0.000000,0.0,0.003401,0.000000,...,0.0,0.003401,0.0,0.862425,0.021496,0.00086,0.090284,0.0,0.024936,0.0
2,1607088730,0,0,1454,262,0.000000,0.007634,0.0,0.000000,0.003817,...,0.0,0.003817,0.0,0.893617,0.003404,0.00000,0.085106,0.0,0.017872,0.0
3,1607088999,0,0,1454,262,0.000000,0.007634,0.0,0.000000,0.003817,...,0.0,0.003817,0.0,0.893617,0.003404,0.00000,0.085106,0.0,0.017872,0.0
4,2128,0,0,880,163,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.881766,0.001425,0.00000,0.089744,0.0,0.027066,0.0


In [14]:
audio = pd.read_csv("scriptedML.csv")
audio = audio.drop(columns = ["phq9","q9"])

data = data.merge(audio, on = "id", how = "inner")
print(data.shape)
data.head()

(115, 2476)


,id,phq9,q9,Characters,Words,help,office,dance,money,wedding,...,logHNR_sma_de_risetime,F0final_sma_f0v_nnz,F0final_sma_f0v_meanSegLen,F0final_sma_f0v_maxSegLen,F0final_sma_f0v_minSegLen,F0final_sma_f0v_segLenStddev,F0final_sma_f0p_meanSegLen,F0final_sma_f0p_maxSegLen,F0final_sma_f0p_minSegLen,F0final_sma_f0p_segLenStddev
0,6831,0,0,1483,295,0.003401,0.000000,0.0,0.003401,0.000000,...,0.468014,0.375839,0.163077,0.88,0.02,0.235090,0.273846,1.16,0.04,0.286747
1,1607088730,0,0,1454,262,0.000000,0.007634,0.0,0.000000,0.003817,...,0.440092,0.663230,0.312500,1.22,0.02,0.353839,0.181000,0.45,0.03,0.168134
2,1608672132,1,1,698,137,0.000000,0.065693,0.0,0.007299,0.007299,...,0.486698,0.534070,0.436923,2.40,0.02,0.595191,0.487000,2.32,0.03,0.652090
3,1609903202,0,0,622,111,0.000000,0.000000,0.0,0.000000,0.009009,...,0.479730,0.338475,0.277692,1.19,0.02,0.373778,0.548461,1.99,0.02,0.638795
4,1607440988,0,0,556,105,0.000000,0.000000,0.0,0.000000,0.028571,...,0.474860,0.544041,0.341111,0.72,0.02,0.296927,0.280000,1.27,0.03,0.381779


In [15]:
#testids = [1607712777, 292, 2613, 1610640355, 1607494599, 1607040811, 1608492986, 6390, 396, 1607734901, 1607350992, 1608992344, 1609903202, 74, 7159, 4698, 7547, 4441, 1607097951, 8479, 8170, 4707, 7516, 1609174124, 1608853150, 8516, 1611424664, 2843, 1607040596, 1953, 1607772081, 1608564004, 2627, 1607217921, 1607118643, 1607314413, 1609887404, 1608335387, 4098, 1607046006, 1608242917, 8918, 1607131299, 9754, 1607262842, 1607273026, 2478, 1607536408, 1607291545, 1608707232, 1609941585, 1608200497, 1610630377, 7711, 1607810287, 9934, 1608850448, 4041, 1609166629, 1608168856, 1607572897, 6831, 1608586814, 1608588581, 2837, 8180, 1608631410, 1607051003, 3830, 4879, 1608920128, 1607019351, 8181, 3473, 1608335906, 1607738757, 1608770486, 7564, 1607495239, 1609983150, 1607397061, 1607696074, 103, 2222]
trainids = [4769, 1607928177, 1607269923, 7755, 4598, 1607807806, 1608741452, 3323, 1610110670, 1607133044, 9745, 1607291670, 5245, 4442, 319, 1607133218, 1607010270, 1608587203, 1609256130, 1608582258, 5028, 1609771771, 5229, 3517, 1608595561, 1608048050, 1607410780, 528, 1607134906, 3102, 1607555727, 1609887167, 3985, 7256, 3523, 1607289708, 1609890222, 850, 1608917024, 5047, 1608061691, 4782, 1608062276, 1056, 1611517276, 1607636681, 1607891972, 5571, 1609052616, 1607927243, 2525, 4353, 1610818662, 8640, 1607559849, 6706, 1608624428, 1607968838, 1608672132, 552, 1608537399, 1610381937, 1608607986, 381, 1608589576, 3920, 1608059746, 1609027319, 1607357022, 1607691623, 1609899907, 1608470962, 8791, 1610380419, 3064, 1609473849, 1607712704, 1609887249, 1609888813, 1608588103, 1244, 7279, 1607339125, 1607712682, 8472, 1269, 1607045076, 1607365865, 1846, 191, 1811, 1608702785, 1609049435, 5330, 1607257348, 1609890530, 3278, 1608586899, 1607939718, 2430, 1609893292, 60, 1607270186, 6336, 8650, 1608495626, 1608586953, 2121, 1607295286, 896, 1609889389, 1607560754, 6548, 6580, 1607440988, 1609111416, 1607807159, 8663, 1607129044, 6658, 1607799213, 3933, 1608596696, 1608663032, 1610791060, 1607135820, 1607413039, 1607659758, 1608487726, 4859, 1609142183, 1607276888, 7452, 1607368510, 1607266081, 2623, 1608416516, 2128, 3227, 5881, 6510, 1609166843, 7569, 1607712793, 1608850996, 3273, 1607939838, 9986, 3302, 1607206195, 1609082904, 1607510222, 7612, 1607022963, 1607051040, 1607719324, 1608849324, 1607642639, 1607104225, 705, 1608506424, 1608188073, 8018, 8085, 4755, 1611704179, 1607193886, 7007, 3041, 4001, 1552, 1716, 1608053349, 1608572299, 1608051417, 1607712784, 836, 1607929944, 1607795480, 1608200317, 415, 3662, 1610109929, 2496, 8550, 6868, 1608587385, 1608591490, 7370, 4549, 7505, 1879, 1876, 1608003341]
data = data[data['id'].isin(trainids)]
data.head()

,id,phq9,q9,Characters,Words,help,office,dance,money,wedding,...,logHNR_sma_de_risetime,F0final_sma_f0v_nnz,F0final_sma_f0v_meanSegLen,F0final_sma_f0v_maxSegLen,F0final_sma_f0v_minSegLen,F0final_sma_f0v_segLenStddev,F0final_sma_f0p_meanSegLen,F0final_sma_f0p_maxSegLen,F0final_sma_f0p_minSegLen,F0final_sma_f0p_segLenStddev
2,1608672132,1,1,698,137,0.000000,0.065693,0.0,0.007299,0.007299,...,0.486698,0.534070,0.436923,2.40,0.02,0.595191,0.487000,2.32,0.03,0.652090
4,1607440988,0,0,556,105,0.000000,0.000000,0.0,0.000000,0.028571,...,0.474860,0.544041,0.341111,0.72,0.02,0.296927,0.280000,1.27,0.03,0.381779
6,1607659758,0,0,479,86,0.000000,0.000000,0.0,0.000000,0.011628,...,0.490783,0.494203,0.251538,0.85,0.03,0.281038,0.207500,1.10,0.02,0.268573
7,1607134906,0,0,394,77,0.012987,0.000000,0.0,0.000000,0.000000,...,0.475961,0.532033,0.471250,1.02,0.02,0.355051,0.538333,2.47,0.02,0.867552
8,4598,0,0,426,76,0.000000,0.000000,0.0,0.000000,0.013158,...,0.484581,0.357289,0.183333,0.55,0.04,0.184932,0.303000,1.02,0.07,0.286742


In [16]:
labels = ["phq9", "q9"]
featureEs = ["Chi", "PCA"]
modelTypelist = ["NB", "XG2", "XG3", "XG4", "LR1", "LR2", "SVC1", "SVC2", "SVC3", "SVC4", "kNN1", "kNN3", "kNN5", "kNN7", "kNN9", "RF2g", "RF2e", "RF3g", "RF3e", "RF4g", "RF4e"]

for label in labels:
    
    #create lists to populate
    flist = [] 
    mlist = []
    llist = []
    featureList = []
    f1List = []
    accuracyList = []
    truePosList = []
    trueNegList = []
    falsePosList = []
    falseNegList = []
    predictions = []
    rseed = []
    
    for r in [1,2,4,8,16,32,64,128,256,512]:

        #train/test split    
        df_train, df_test = train_test_split(data, test_size=0.3, stratify=data[["phq9", "q9"]], random_state = r)
        trainids = list(df_train["id"])
        testids = list(df_test["id"])
        print(data.shape)
        testdata = data[data['id'].isin(testids)]
        print(testdata.shape)
        traindata = data[data['id'].isin(trainids)]
        print(traindata.shape)

        #limit to features
        testContent = testdata[testdata.columns[3:]]
        print(testContent.shape)
        trainContent = traindata[traindata.columns[3:]]
        print(trainContent.shape)

        #NEED TO SCALE BEFORE FEATURE SELECTION/REDUCATION
        min_max_scaler = preprocessing.MinMaxScaler()  
        np_scaled = min_max_scaler.fit_transform(trainContent)
        featureSubset = pd.DataFrame(np_scaled)
        np_scaled2 =  min_max_scaler.transform(testContent)
        testSubset = pd.DataFrame(np_scaled2)
        print(featureSubset.shape)
        print(testSubset.shape)

        target = list(traindata[label])

        for featureE in featureEs:

            featureDF = []
            testDFs = []

            if featureE == "Chi":
                nFeatureList = list(np.arange(1,11,1))

                for numberOfFeatures in nFeatureList:
                    chisetup = SelectKBest(chi2, k=numberOfFeatures)
                    chisetup = chisetup.fit(featureSubset, target)
                    featureSubset2 = chisetup.transform(featureSubset)
                    featureSubset2=pd.DataFrame(featureSubset2).assign(target = target)
                    featureDF.append(featureSubset2)
                    testSubset2 = chisetup.transform(testSubset)
                    testDFs.append(pd.DataFrame(testSubset2))

            elif featureE == "PCA":
                nFeatureList = list(np.arange(1,11,1))
                for numberOfFeatures in nFeatureList:
                    pca = PCA(n_components=numberOfFeatures)
                    pca = pca.fit(featureSubset)
                    X_pca = pca.transform(featureSubset)
                    pcaDF = pd.DataFrame(X_pca)
                    pcaDF = pcaDF.assign(target = target)
                    featureDF.append(pcaDF)
                    testSubset2 = pca.transform(testSubset)
                    testDFs.append(pd.DataFrame(testSubset2))


            for f in range(0, len(featureDF)):
                print(featureE)
                print(f)

                train_phq9 = featureDF[f]
                X_test = testDFs[f]

                # upsampling 
                #Count 1s and 0s
                ones = len(train_phq9.loc[train_phq9['target'] == 1])
                zeros = len(train_phq9.loc[train_phq9['target'] == 0])
                if ones >= zeros:
                    majority = 1
                    minority = 0
                else:
                    majority = 0
                    minority = 1


                # Upsample TrainingSet 
                train_majority = train_phq9[train_phq9.target==majority]
                train_minority = train_phq9[train_phq9.target==minority]

                #print("train_majority ="  + str(len(train_majority)))
                #print("train_minority ="  + str(len(train_minority)))

                # Upsample minority class
                train_minority_upsampled = resample(train_minority, 
                                                 replace=True,     # sample with replacement
                                                 n_samples=len(train_majority),    # to match majority class
                                                 random_state=42) # reproducible results

                # Combine majority class with upsampled minority class
                train_phq9 = pd.concat([train_majority, train_minority_upsampled])

                #seperate features and target
                y_train = train_phq9["target"]
                X_train = train_phq9.drop(columns = "target")


                print(X_train.head())
                print(X_test.head())
            
                for modelType in modelTypelist:

                    #add data to lists
                    llist.append(label)
                    featureList.append(f +1)
                    flist.append(featureE)
                    mlist.append(modelType)

                    #chose model type
                    if modelType == "SVC1":
                        clf = svm.SVC(kernel='rbf', random_state=r)
                    elif modelType == "SVC2":
                        clf = svm.SVC(kernel='linear', random_state=r)
                    elif modelType == "SVC3":
                        clf = svm.SVC(kernel='poly', random_state=r)
                    elif modelType == "SVC4":
                        clf = svm.SVC(kernel='sigmoid', random_state=r)
                    elif modelType == "RF2g":
                        clf = RandomForestClassifier(criterion="gini", max_depth=2, random_state=r)
                    elif modelType == "RF2e":
                        clf = RandomForestClassifier(criterion="entropy", max_depth=2, random_state=r)
                    elif modelType == "RF3g":
                        clf = RandomForestClassifier(criterion="gini", max_depth=3, random_state=r)
                    elif modelType == "RF3e":
                        clf = RandomForestClassifier(criterion="entropy", max_depth=3, random_state=r)
                    elif modelType == "RF4g":
                        clf = RandomForestClassifier(criterion="gini", max_depth=4, random_state=r)
                    elif modelType == "RF4e":
                        clf = RandomForestClassifier(criterion="entropy", max_depth=4, random_state=r)
                    elif modelType == "kNN1":
                        clf = KNeighborsClassifier(n_neighbors=1)
                    elif modelType == "kNN3":
                        clf = KNeighborsClassifier(n_neighbors=3)
                    elif modelType == "kNN5":
                        clf = KNeighborsClassifier(n_neighbors=5)
                    elif modelType == "kNN7":
                        clf = KNeighborsClassifier(n_neighbors=7)
                    elif modelType == "kNN9":
                        clf = KNeighborsClassifier(n_neighbors=9)
                    elif modelType == "XG2":
                        clf = xgb.XGBClassifier(max_depth=2, random_state=r)
                    elif modelType == "XG3":
                        clf = xgb.XGBClassifier(max_depth=3, random_state=r)
                    elif modelType == "XG4":
                        clf = xgb.XGBClassifier(max_depth=4, random_state=r)
                    elif modelType == "LR1":
                        clf = LogisticRegression(random_state=r)
                    elif modelType == "LR2":
                        clf = LogisticRegression(random_state=r, penalty = "l2")
                    elif modelType == "NB":
                        clf = GaussianNB()


                    #train model and make predictions
                    clf.fit(X_train, y_train)
                    y_pred = clf.predict(X_test)

                    #evaluate model
                    conf_mat = confusion_matrix(list(testdata[label]), y_pred)
                    TN = conf_mat[0][0]
                    TP = conf_mat[1][1]
                    FP = conf_mat[0][1]
                    FN = conf_mat[1][0]
                    precision = TP/(TP+FP)
                    sensitivity = TP/(TP+FN)
                    f1 = (2*precision*sensitivity)/(precision + sensitivity)
                    accuracy = (TP+TN)/(TN+TP+FP+FN)

                    #populate lists with results
                    f1List.append(f1)
                    accuracyList.append(accuracy)
                    truePosList.append(TP)
                    trueNegList.append(TN)
                    falsePosList.append(FP)
                    falseNegList.append(FN)
                    predictions.append(y_pred)
                    rseed.append(r)

    resultsDF = pd.DataFrame()
    resultsDF["label"] = llist
    resultsDF["Engineering"] = flist
    resultsDF["model"] = mlist
    resultsDF["nFeatures"] = featureList
    resultsDF["F1"] = f1List
    resultsDF["Accuracy"] = accuracyList
    resultsDF["truePos"] = truePosList
    resultsDF["trueNeg"] = trueNegList
    resultsDF["falsePos"] = falsePosList
    resultsDF["falseNeg"] = falseNegList
    resultsDF["predictions"] = predictions
    resultsDF["randomSeed"] = rseed

    resultsDF.to_csv("resultsAll85/parametersScriptedText" + label + ".csv")

(69, 2476)
(21, 2476)
(48, 2476)
(21, 2473)
(48, 2473)
(48, 2473)
(21, 2473)
Chi
0
          0
0  0.000000
1  0.526316
2  0.000000
3  0.000000
4  0.000000
          0
0  0.000000
1  0.285714
2  0.116279
3  0.000000
4  0.208333


C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: RuntimeWarni

Chi
1
     0         1
0  0.0  0.000000
1  0.0  0.526316
2  0.0  0.000000
3  0.0  0.000000
4  0.0  0.000000
     0         1
0  0.0  0.000000
1  0.0  0.285714
2  0.0  0.116279
3  0.0  0.000000
4  0.0  0.208333


C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: RuntimeWarni

Chi
2
     0         1    2
0  0.0  0.000000  0.0
1  0.0  0.526316  0.0
2  0.0  0.000000  0.0
3  0.0  0.000000  0.0
4  0.0  0.000000  0.0
     0         1         2
0  0.0  0.000000  0.051095
1  0.0  0.285714  0.000000
2  0.0  0.116279  0.081395
3  0.0  0.000000  0.000000
4  0.0  0.208333  0.000000


...

C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:186: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: 

PCA
8
          0         1         2         3         4         5         6  \
1 -3.389508 -2.757741  4.228941 -4.738938 -1.489032 -0.207555 -2.782297   
2 -4.858836 -4.305935 -4.542930 -0.248869  3.491964  6.000219  0.960253   
3 -3.415972 -0.631054 -1.572289  1.737409 -1.158666  0.251344  2.531216   
4 -4.459139 -1.838655  4.564843  1.416418  1.341701 -2.253150  2.051735   
5  7.866430  0.242428  0.341054 -1.852624  3.363675 -0.962521  0.336389   

          7         8  
1 -1.158973 -1.071109  
2 -3.583266  1.564059  
3 -0.743542  0.846566  
4 -2.839008  0.904796  
5 -2.436120 -3.049997  
          0          1         2         3         4         5         6  \
0  6.924604  -0.688059 -0.001991 -1.101289  2.143969  0.166879  1.193307   
1 -4.294339  10.935418 -2.084971 -1.771464  1.274110 -0.058952  3.545620   
2 -0.037125   1.466687  0.962633 -0.153265 -0.837850 -0.633695 -0.751531   
3 -1.421621  -1.208722  5.759932 -0.900071 -1.149801 -0.376856  0.623694   
4 -2.909598  -1.288

C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: inva

PCA
9
          0         1         2         3         4         5         6  \
1 -3.389543 -2.757466  4.229465 -4.743761 -1.457509 -0.150674 -2.765143   
2 -4.858805 -4.306016 -4.542944 -0.249854  3.488170  6.025791  0.980125   
3 -3.415935 -0.630888 -1.571227  1.733109 -1.158607  0.265936  2.535728   
4 -4.459152 -1.838249  4.565519  1.414578  1.342449 -2.296888  2.024836   
5  7.866575  0.243069  0.342218 -1.860150  3.418648 -0.789540  0.263176   

          7         8         9  
1 -1.051620 -0.735936  4.245662  
2 -3.519261  1.340427  3.253511  
3 -0.918954  0.682101 -1.965510  
4 -2.732264  0.833470 -1.032601  
5 -2.224895 -2.825478 -0.704019  
          0          1         2         3         4         5         6  \
0  6.924601  -0.687567 -0.002602 -1.101524  2.139826  0.147991  1.200857   
1 -4.294286  10.936659 -2.084738 -1.772886  1.272748 -0.093155  3.515472   
2 -0.037079   1.466828  0.963596 -0.156629 -0.820185 -0.596877 -0.727697   
3 -1.421584  -1.208233  5.759740 -0

C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: inva